# Test the filter time for binary files in azimuth, height, distance format

In [178]:
import numba
from numba import njit, prange
import os
import random
import sys
import time
import numpy as np
import pandas as pd
from pathlib import Path
from contextlib import redirect_stdout
from mmdet3d.apis import LidarDet3DInferencer

## May need to change back the lon values in the numba library
File ~/miniconda3/envs/openmm_mmvc/lib/python3.8/site-packages/numba/core/types/__init__.py:110<br>
    108 # long_ = _make_signed(np.long)<br>
    109 long_ = _make_signed(np.int_)<br>
--> 110 ulong = _make_unsigned(np.long)<br>
    111 longlong = _make_signed(np.longlong)

In [179]:
# Initialize inferencer
inferencer = LidarDet3DInferencer('pointpillars_kitti-3class')

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


/home/rachel/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/home/rachel/miniconda3/envs/openmm_mmvc/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [180]:
azimuth_resolution = 0.05
height_resolution = 0.25

In [181]:
def convert_to_dataframe(bin_path):
    pre_filtered_data = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4) 
    columns = ['azimuth', 'height', 'distance', 'intensity']
    df = pd.DataFrame(pre_filtered_data, columns=columns)
    return df

In [182]:
# Returns list of dataframes of LiDAR points
def get_frames(source_dir, num_frames):
    frame_list = []
    
    # Get frame names
    files = [f for f in os.listdir(source_dir) if f.endswith('.bin')]
    # Shuffle and take desired number of files
    random.shuffle(files)
    files = files[:num_frames]
    
    # For each frame
    for file in files:
        print('.', end='')
        file_path = Path(source_dir, file)
        # Load each frame
        frame_list.append(convert_to_dataframe(file_path))
        
    return frame_list

In [183]:
# Unpickle background map dataframe and fill the empty values
def get_filter(background_map_path):
    background_distance_lookup_table = pd.read_pickle(background_map_path)
    filled_lookup_table = background_distance_lookup_table.ffill(axis=0).bfill(axis=0)
    filled_lookup_table = filled_lookup_table.ffill(axis=1).bfill(axis=1)
    return filled_lookup_table

In [184]:
def add_lookup_coords_to_ahd(points_df):
    # print(points_df)
    # Calculate the distance, azimuth, and height using vectorized operations
    azimuth, height, distance, intensity = points_df['azimuth'], points_df['height'], points_df['distance'], points_df['intensity']
    
    # Convert and scale
    azimuth_idx = (np.floor((azimuth + 180) / azimuth_resolution) * azimuth_resolution * 100 - 18000).astype(int)
    height_idx = (np.floor((height + 30) / height_resolution) * height_resolution * 100 - 3000).astype(int)

    points_df['azimuth_idx'] = azimuth_idx
    points_df['height_idx'] = height_idx
    
    return points_df

In [185]:
def filter_frame(pre_filtered_points, lookup_table):
    # Add lookup table coordinates
    pre_filtered_grid_lookup = add_lookup_coords_to_ahd(pre_filtered_points)
    
    # Set index to ['height_idx', 'azimuth_idx']
    pre_filtered_grid_lookup_indexed = pre_filtered_grid_lookup.set_index(['height_idx', 'azimuth_idx'])
    
    # Create a Series from the lookup table with a MultiIndex
    lookup_series = lookup_table.stack()
    
    # Reindex the lookup values to align with the DataFrame's index
    lookup_values = lookup_series.reindex(pre_filtered_grid_lookup_indexed.index)
    
    # Create a mask for indices that exist in the lookup_table
    indices_in_lookup = lookup_values.index.isin(lookup_series.index)
    
    # Create the condition based on filtering criteria
    condition = (
        indices_in_lookup & (
            lookup_values.isna() | 
            (pre_filtered_grid_lookup_indexed['distance'] < lookup_values)
        )
    )
    
    # Apply the condition to filter the DataFrame
    filtered_df = pre_filtered_grid_lookup_indexed[condition].reset_index()
    
    return filtered_df

In [186]:
def convert_to_xyz(points_df):
    azimuth, height, distance, intensity = points_df['azimuth'], points_df['height'], points_df['distance'], points_df['intensity']
    
    azimuth_rad = np.deg2rad(azimuth)
    height_rad = np.deg2rad(height)
    x = distance * np.cos(height_rad) * np.sin(azimuth_rad)
    y = distance * np.cos(height_rad) * np.cos(azimuth_rad)
    z = distance * np.sin(height_rad)
    
    # Stack the computed values into a numpy array
    xyz_intensity_array = np.column_stack((x, y, z, intensity))
    
    return xyz_intensity_array

In [187]:
def time_filter(lidar_frames_list, filter_df, filter_points=True):
    # Start timer
    start = time.time()
    # For each frame
    for frame_df in lidar_frames_list:
        # Filter frame
        if filter_points:
            frame_df = filter_frame(frame_df, filter_df)
        # Convert to x, y, z
        xyz_frame = convert_to_xyz(frame_df)
        
        # Run Inferences
        inferencer(dict(points=xyz_frame))
    # Stop timer
    end = time.time()
    return end - start

In [188]:
def run_test(data_dir, background_map_path, num_frames):
    # Get dataframes
    lidar_frames_list = get_frames(data_dir, num_frames)
    filter_df = get_filter(background_map_path)
    
    total_time_filter = time_filter(lidar_frames_list, filter_df)
    
    total_time_no_filter = time_filter(lidar_frames_list, pd.DataFrame(), filter_points=False)
    
    results_filter = {
        'type': 'filtered',
        'num_frames': num_frames,
        'total_time': total_time_filter,
        'time_per_frame': total_time_filter / num_frames
    }
    
    results_no_filter = {
        'type': 'not filtered',
        'num_frames': num_frames,
        'total_time': total_time_no_filter,
        'time_per_frame': total_time_no_filter / num_frames
    }
    return [results_filter, results_no_filter]

In [189]:
data_dir = '../data/az_height_dist_points'
background_map_path = '../data/lookup_table.pkl'

In [190]:
%%capture
# Run time test and get results
time_results = run_test(data_dir, background_map_path, 200)

In [191]:
results_df = pd.DataFrame(time_results)
display(results_df)

,type,num_frames,total_time,time_per_frame
0,filtered,200,34.681064,0.173405
1,not filtered,200,17.203532,0.086018


# Numba versions of functions

In [192]:
def convert_to_np_array(bin_path):
    pre_filtered_data = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4) 
    return pre_filtered_data

In [193]:
# Returns list of frames of LiDAR points
def get_frames(source_dir, num_frames):
    frame_list = []
    
    # Get frame names
    files = [f for f in os.listdir(source_dir) if f.endswith('.bin')]
    # Shuffle and take desired number of files
    random.shuffle(files)
    files = files[:num_frames]
    
    # For each frame
    for file in files:
        print('.', end='')
        file_path = Path(source_dir, file)
        # Load each frame
        frame_list.append(convert_to_np_array(file_path))
        
    return frame_list

In [194]:
# Unpickle background map dataframe and fill the empty values
def get_filter(background_map_path):
    background_distance_lookup_table = pd.read_pickle(background_map_path)
    filled_lookup_table = background_distance_lookup_table.ffill(axis=0).bfill(axis=0)
    filled_lookup_table = filled_lookup_table.ffill(axis=1).bfill(axis=1)
    return filled_lookup_table

In [195]:
@njit
def compute_lookup_coords(azimuth, height):
    azimuth_idx = np.floor((azimuth + 180) / azimuth_resolution) * azimuth_resolution * 100 - 18000
    height_idx = np.floor((height + 30) / height_resolution) * height_resolution * 100 - 3000

    azimuth_idx = azimuth_idx.astype(np.int32)
    height_idx = height_idx.astype(np.int32)

    return azimuth_idx, height_idx

In [196]:
def add_lookup_coords_to_ahd(np_frame):
    # Extract columns from np_frame
    azimuth = np_frame[:, 0]
    height = np_frame[:, 1]
    distance = np_frame[:, 2]
    intensity = np_frame[:, 3]

    # Compute indices using the Numba-accelerated function
    azimuth_idx, height_idx = compute_lookup_coords(azimuth, height)

    # Stack the new indices onto the original data
    # The result will be a NumPy array with shape (n_samples, 6)
    augmented_frame = np.column_stack((azimuth, height, distance, intensity, azimuth_idx, height_idx))

    return augmented_frame

In [197]:
@njit(parallel=True)
def filter_points_numba(distance, azimuth_idx, height_idx, lookup_values_array, height_indices, azimuth_indices):
    num_points = len(distance)
    filtered_mask = np.zeros(num_points, dtype=np.bool_)

    for i in prange(num_points):
        h_idx = height_idx[i]
        a_idx = azimuth_idx[i]

        # Find positions using searchsorted
        h_pos = np.searchsorted(height_indices, h_idx)
        a_pos = np.searchsorted(azimuth_indices, a_idx)

        # Check if indices are within bounds and match the desired indices
        if h_pos < len(height_indices) and height_indices[h_pos] == h_idx and \
           a_pos < len(azimuth_indices) and azimuth_indices[a_pos] == a_idx:
            lookup_value = lookup_values_array[h_pos, a_pos]
            if not np.isnan(lookup_value):
                if distance[i] < lookup_value:
                    filtered_mask[i] = True
            else:
                filtered_mask[i] = True  # Include if lookup_value is NaN
        else:
            # Indices not found in lookup table; decide whether to include or exclude
            filtered_mask[i] = False  # Exclude by default

    return filtered_mask

In [198]:
def filter_frame(np_frame, lookup_table):
    # Add lookup coordinates
    augmented_frame = add_lookup_coords_to_ahd(np_frame)

    # Extract the necessary columns
    distance = augmented_frame[:, 2]
    azimuth_idx = augmented_frame[:, 4].astype(np.int32)
    height_idx = augmented_frame[:, 5].astype(np.int32)

    # Convert the lookup table to NumPy arrays
    lookup_values_array = lookup_table.values.copy()
    height_indices = lookup_table.index.values.copy()
    azimuth_indices = lookup_table.columns.values.copy()

    # Make sure the indices are sorted and reorder the lookup values accordingly
    height_sort_order = np.argsort(height_indices)
    height_indices = height_indices[height_sort_order]
    lookup_values_array = lookup_values_array[height_sort_order, :]

    azimuth_sort_order = np.argsort(azimuth_indices)
    azimuth_indices = azimuth_indices[azimuth_sort_order]
    lookup_values_array = lookup_values_array[:, azimuth_sort_order]

    # Apply the Numba filtering function
    filtered_mask = filter_points_numba(distance, azimuth_idx, height_idx,
                                        lookup_values_array, height_indices, azimuth_indices)

    # Apply the mask to the original data
    filtered_data = augmented_frame[filtered_mask]
    return filtered_data

In [199]:
@njit
def convert_to_xyz_numba(azimuth, height, distance, intensity):
    # Convert degrees to radians
    azimuth_rad = np.deg2rad(azimuth)
    height_rad = np.deg2rad(height)

    # Precompute trigonometric functions
    cos_height = np.cos(height_rad)
    sin_height = np.sin(height_rad)
    sin_azimuth = np.sin(azimuth_rad)
    cos_azimuth = np.cos(azimuth_rad)

    # Compute x, y, z
    x = distance * cos_height * sin_azimuth
    y = distance * cos_height * cos_azimuth
    z = distance * sin_height

    # Stack the computed values into a numpy array
    xyz_intensity_array = np.column_stack((x, y, z, intensity))

    return xyz_intensity_array

In [200]:
def convert_to_xyz(np_frame):
    # Extract columns from np_frame
    azimuth = np_frame[:, 0]
    height = np_frame[:, 1]
    distance = np_frame[:, 2]
    intensity = np_frame[:, 3]

    # Call the Numba-accelerated function
    xyz_intensity_array = convert_to_xyz_numba(azimuth, height, distance, intensity)

    return xyz_intensity_array

In [201]:
def save_as_binary(np_array, bin_path):
    try:
        # Ensure the array is in float32 type
        data = np_array.astype(np.float32)
        
        # Write the data to a binary file
        data.tofile(bin_path)
    except Exception as e:
        print('Could not save:', bin_path)
        print('Error:', e)

In [202]:
test_dir = '../data/test'
def time_filter(lidar_frames_list, filter_df, filter_points=True):
    # Start timer
    start = time.time()
    # For each frame
    for index, frame_np in enumerate(lidar_frames_list):
        # Filter frame
        if filter_points:
            frame_np = filter_frame(frame_np, filter_df)
        # Convert to x, y, z
        xyz_frame = convert_to_xyz(frame_np)
        
        # Run Inferences
        inferencer(dict(points=xyz_frame))
    # Stop timer
    end = time.time()
    return end - start

In [203]:
def run_test(data_dir, background_map_path, num_frames):
    # Get dataframes
    lidar_frames_list = get_frames(data_dir, num_frames)
    filter_df = get_filter(background_map_path)
    
    total_time_filter = time_filter(lidar_frames_list, filter_df)
    
    total_time_no_filter = time_filter(lidar_frames_list, pd.DataFrame(), filter_points=False)
    
    results_filter = {
        'type': 'filtered',
        'num_frames': num_frames,
        'total_time': total_time_filter,
        'time_per_frame': total_time_filter / num_frames
    }
    
    results_no_filter = {
        'type': 'not filtered',
        'num_frames': num_frames,
        'total_time': total_time_no_filter,
        'time_per_frame': total_time_no_filter / num_frames
    }
    return [results_filter, results_no_filter]

In [204]:
data_dir = '../data/az_height_dist_points'
background_map_path = '../data/lookup_table.pkl'

In [205]:
%%capture
# Run time test and get results
time_results = run_test(data_dir, background_map_path, 200)

In [206]:
results_df = pd.DataFrame(time_results)
display(results_df)

,type,num_frames,total_time,time_per_frame
0,filtered,200,16.605084,0.083025
1,not filtered,200,18.230908,0.091155
